In [413]:
import pandas as pd
import json

Per la realizzazione ho seguito https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

In [414]:
dataset = pd.read_csv('description_predictions.csv')
with open("sinonimi.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [415]:
dataset_withoutNAN = dataset[dataset.prediction != 'NAN']

range_index = [i for i in range(len(dataset_withoutNAN))]
dataset_withoutNAN['index'] = range_index 

dataset_withoutNAN = dataset_withoutNAN.set_index('index') 

<ipython-input-415-3e36f7bd9202>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_withoutNAN['index'] = range_index


Il codice seguente serve per convertire i soprannomi delle squadre in home team e away team

In [416]:
"""for i, row in dataset_withoutNAN.iterrows():
    h_team, a_team, description, prediction = row.h_team, row.a_team, row.description, row.prediction

    syn = {}
    #cerco nel dizionario di sinonimi, tutti i sinonimi delle squadre del match
    for key in jsonObject.keys():
        if (h_team in key) or (key in h_team):
            syn['home team'] = jsonObject[key] 
            
        if (a_team in key) or (key in a_team):
            syn['away team'] = jsonObject[key] 

    #successivamente prendo il testo e sostituisco i sinonimi con home o away team
    description = description.lower()
    for key in syn.keys():
        for val in syn[key]:
            description = description.replace(val.lower(), key)

    dataset_withoutNAN.at[i, 'description'] = description


In [417]:
dataset_withoutNAN = dataset_withoutNAN[['description', 'prediction']]
prediction_labels = {
    'N': 0,
    'V': 1,
    'P': 2
}

dataset_withoutNAN['prediction'] = dataset_withoutNAN['prediction'].map(prediction_labels)
dataset_withoutNAN.head()

,description,prediction
index,,
0,home team and away team take on each other in ...,0
1,home team begin their title defence with a mat...,1
2,away team will look to make a flying start to ...,2
3,newcomers home team will be aiming to kick off...,2
4,after failing to impress in the 2020/21 serie ...,2


In [418]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True, stop_words='english', ngram_range = (2,2), tokenizer = token.tokenize)
text_counts= cv.fit_transform(dataset_withoutNAN.description)

In [419]:
text_counts.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1]])

In [420]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, dataset_withoutNAN.prediction, test_size=0.3, random_state=1)

In [421]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as mtr


models = {
    'Logistic Regression': LogisticRegression(max_iter=10000), #max_iter di default vale 100, ho dovuto alzarlo se no non converge
    'Support Vector Machine': SVC(),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

In [422]:
predictions = {}
for model_name, model in models.items():
    models[model_name].fit(X_train, y_train)
    predictions[model_name] = model.predict(X_test)

E = []
for estimator, y_pred in predictions.items():
    report = mtr.classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    E.append({
        'Model': estimator, 'Accuracy': report['accuracy'],
        'Avg Precision (macro)': report['macro avg']['precision'],
        'Avg Recall (macro)': report['macro avg']['recall'],
        'Avg F1-score (macro)': report['macro avg']['f1-score'],
        'Avg Precision (weighted)': report['weighted avg']['precision'],
        'Avg Recall (weighted)': report['weighted avg']['recall'],
        'Avg F1-score (weighted)': report['weighted avg']['f1-score']
    })
E = pd.DataFrame(E).set_index('Model', inplace=False)

In [423]:
E

,Accuracy,Avg Precision (macro),Avg Recall (macro),Avg F1-score (macro),Avg Precision (weighted),Avg Recall (weighted),Avg F1-score (weighted)
Model,,,,,,,
Logistic Regression,0.666667,0.333333,0.500000,0.400000,0.444444,0.666667,0.533333
Support Vector Machine,0.666667,0.333333,0.500000,0.400000,0.444444,0.666667,0.533333
Multinomial Naive Bayes,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Decision Tree,0.666667,0.333333,0.500000,0.400000,0.444444,0.666667,0.533333
Random Forest,0.666667,0.333333,0.500000,0.400000,0.444444,0.666667,0.533333
K-Nearest Neighbors,0.333333,0.333333,0.166667,0.222222,0.666667,0.333333,0.444444
